In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import json
from underthesea import word_tokenize

from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import datasets
device = "cuda" if torch.cuda.is_available() else "cpu"


import re

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

from tqdm import tqdm

In [2]:
df = pd.read_csv("data/train.csv")

In [3]:
df = df[df["evidence"].notna()]

In [4]:
df["evidence_start"] = df.apply(lambda x: x["context"].find(x["evidence"]), axis=1)
df["evidence_end"] = df.apply(lambda x: x["evidence_start"] + len(x["evidence"]), axis=1)

In [5]:
from transformers import PreTrainedTokenizerFast, AutoModelForQuestionAnswering, AutoTokenizer
model = AutoModelForQuestionAnswering.from_pretrained("nguyenvulebinh/vi-mrc-base")
# model.to(device)
tokenizer = AutoTokenizer.from_pretrained("nguyenvulebinh/vi-mrc-base")
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [6]:
df_train, df_val = train_test_split(df, test_size=0.05, random_state=42)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

In [7]:
train_dataset = datasets.Dataset.from_pandas(df_train)
val_dataset = datasets.Dataset.from_pandas(df_val)

In [8]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["claim"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
        
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["evidence"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = examples["evidence_start"][i]
        end_char = examples["evidence_end"][i]
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [9]:
train_datasets = train_dataset.map(preprocess_function,batched=True, batch_size=32,remove_columns=train_dataset.column_names)
valid_datasets = val_dataset.map(preprocess_function,batched=True, batch_size=32,remove_columns=val_dataset.column_names)

Map:   0%|          | 0/23738 [00:00<?, ? examples/s]

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

In [10]:
sample = train_datasets[10]
envidence = sample["input_ids"][sample["start_positions"]:sample["end_positions"]]

In [11]:
train_dataset[10]

{'context': 'chủ tịch quốc hội vương đình huệ thăm chính thức uruguay đến ngày 28/4, theo lời mời của chủ tịch thượng viện nước này beatriz argimon cedeira. đây là chuyến thăm cấp cao đầu tiên của việt nam tới uruguay, vào dịp kỷ niệm 30 năm thiết lập quan hệ ngoại giao giữa hai nước.\n\ntrong chuyến thăm, chủ tịch quốc hội vương đình huệ dự kiến hội đàm, hội kiến với lãnh đạo uruguay, ký thỏa thuận hợp tác giữa cơ quan lập pháp hai nước, dự một số hoạt động kỷ niệm 30 năm thiết lập quan hệ ngoại giao việt nam - uruguay.\n\nviệt nam và uruguay thiết lập quan hệ ngoại giao ngày 11/8/1993. trong 30 năm qua, quan hệ hữu nghị và hợp tác giữa hai nước đã phát triển, trao đổi thương mại hai chiều tăng từ 27 triệu usd năm 2007 lên hơn 100 triệu usd năm 2019. do tác động của đại dịch covid-19, trao đổi thương mại giảm phần nào, nhưng sang năm 2022, kim ngạch hai chiều đã tăng gần 90% so với năm 2021, đạt hơn 175 triệu usd.\n\nhai nước ủng hộ lẫn nhau tại các diễn đàn khu vực và quốc tế, phát h

In [12]:
tokenizer.decode(envidence)

'</s> chủ tịch quốc hội vương đình huệ thăm chính thức uruguay đến ngày 28/4, theo lời mời của chủ tịch thượng viện nước này beatriz argimon cedeira. đây'

In [13]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [14]:
training_args = TrainingArguments(
    output_dir="models/evidence",
    num_train_epochs=10,              # total number of training epochs
    learning_rate=1e-5,              # learning rate
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    # gradient_accumulation_steps=2,   # Number of updates steps to accumulate before performing a backward/update pass.
    warmup_steps=250,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=250,
    eval_steps=250,
    evaluation_strategy='steps',
    load_best_model_at_end=True,
    greater_is_better=True,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_datasets,
    eval_dataset=valid_datasets,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/media/bbsw/Data1/Hung-ws/lazy/v2/.venv/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
250,3.926900,1.928491
500,1.844500,1.792933
750,1.758700,1.760567
1000,1.645300,1.602615
1250,1.586900,1.554161
1500,1.543000,1.523248


/media/bbsw/Data1/Hung-ws/lazy/v2/.venv/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/media/bbsw/Data1/Hung-ws/lazy/v2/.venv/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/media/bbsw/Data1/Hung-ws/lazy/v2/.venv/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


KeyboardInterrupt: 

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
model = AutoModelForQuestionAnswering.from_pretrained("/media/bbsw/Data1/Hung-ws/lazy/v2/models/evidence/checkpoint-500")
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("/media/bbsw/Data1/Hung-ws/lazy/v2/models/evidence/checkpoint-500")

In [15]:
claim = df_val["claim"][10]
context = df_val["context"][10]
evidence = df_val["evidence"][10]


In [16]:
inputs = tokenizer.encode_plus(claim, context, return_tensors="pt", truncation=True).to(device)
with torch.no_grad():
    outputs = model(**inputs)

In [17]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

In [18]:
answer_start_index,answer_end_index

(tensor(28, device='cuda:0'), tensor(53, device='cuda:0'))

In [19]:
predict_answer_tokens =inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

'</s> tại buổi tiếp xúc công nhân với đại biểu quốc hội ngày 12/5, ông nguyễn văn tân, chủ tịch công đoàn'

In [20]:
evidence

'Danh sách sau đó công khai trên cổng BHXH tỉnh hoặc phạt hành chính tùy mức độ chậm trễ của doanh nghiệp'

In [ ]:
answer_start_index, answer_end_index

(tensor(12, device='cuda:0'), tensor(93, device='cuda:0'))